In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('./Data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

./Data/adult.csv


### Init data

In [9]:
!pip install seaborn
import matplotlib.pyplot as plt
import seaborn as sns


In [10]:
data = pd.read_csv('./Data/adult.csv')
df = data.copy()
df.head(10)

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K
5,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K
6,29,?,227026,HS-grad,9,Never-married,?,Unmarried,Black,Male,0,0,40,United-States,<=50K
7,63,Self-emp-not-inc,104626,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103,0,32,United-States,>50K
8,24,Private,369667,Some-college,10,Never-married,Other-service,Unmarried,White,Female,0,0,40,United-States,<=50K
9,55,Private,104996,7th-8th,4,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,10,United-States,<=50K


#### dataset overview

In [11]:
print(df.shape)
display(df.info())

(48842, 15)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        48842 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       48842 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   48842 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


None

Missing values : '?' indicates missing values in the dataset.

In [12]:
display(df.isin(['?']).sum())

age                   0
workclass          2799
fnlwgt                0
education             0
educational-num       0
marital-status        0
occupation         2809
relationship          0
race                  0
gender                0
capital-gain          0
capital-loss          0
hours-per-week        0
native-country      857
income                0
dtype: int64

education and educational-num both indicate the education level

In [13]:
grouped_data = df.groupby(['native-country', 'income']).size().unstack()

# Normalize the counts to get percentages if needed
grouped_data = grouped_data.div(grouped_data.sum(axis=1), axis=0)

# Plotting
plt.figure(figsize=(12, 8))
print(grouped_data)


income                         <=50K      >50K
native-country                                
?                           0.743291  0.256709
Cambodia                    0.678571  0.321429
Canada                      0.653846  0.346154
China                       0.704918  0.295082
Columbia                    0.952941  0.047059
Cuba                        0.753623  0.246377
Dominican-Republic          0.951456  0.048544
Ecuador                     0.866667  0.133333
El-Salvador                 0.929032  0.070968
England                     0.629921  0.370079
France                      0.578947  0.421053
Germany                     0.718447  0.281553
Greece                      0.632653  0.367347
Guatemala                   0.965909  0.034091
Haiti                       0.880000  0.120000
Holand-Netherlands          1.000000       NaN
Honduras                    0.900000  0.100000
Hong                        0.733333  0.266667
Hungary                     0.684211  0.315789
India        

<Figure size 1200x800 with 0 Axes>

In [15]:
df['native-country'].value_counts()

native-country
United-States                 43832
Mexico                          951
?                               857
Philippines                     295
Germany                         206
Puerto-Rico                     184
Canada                          182
El-Salvador                     155
India                           151
Cuba                            138
England                         127
China                           122
South                           115
Jamaica                         106
Italy                           105
Dominican-Republic              103
Japan                            92
Guatemala                        88
Poland                           87
Vietnam                          86
Columbia                         85
Haiti                            75
Portugal                         67
Taiwan                           65
Iran                             59
Greece                           49
Nicaragua                        49
Peru         

- Country : 
Holand-Netherlands has only one entry. 
We can't split it randomly and train an encoder.

In [16]:
print(len(df['educational-num'].unique()))
print(len(df['education'].unique()))

16
16


In [17]:
education_mapping = data.groupby('education')['educational-num'].unique()
print(education_mapping.sort_values())

education
Preschool        [1]
1st-4th          [2]
5th-6th          [3]
7th-8th          [4]
9th              [5]
10th             [6]
11th             [7]
12th             [8]
HS-grad          [9]
Some-college    [10]
Assoc-voc       [11]
Assoc-acdm      [12]
Bachelors       [13]
Masters         [14]
Prof-school     [15]
Doctorate       [16]
Name: educational-num, dtype: object


edutcation and educationnal-num bring the same information : education inducate the level of education, educational-num indicate the years

we won't remove capital gain and capital loss. But their values are to variant so we need to handle more outlying values

In [18]:
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, RobustScaler, StandardScaler, FunctionTransformer
from sklearn.impute import SimpleImputer, KNNImputer

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import GridSearchCV

### Setup dataset

In [19]:
df = df.replace('?', np.nan)

y = df['income']
X = df.drop(['income', 'educational-num'], axis=1)

num_cols = X.select_dtypes(include=np.number).columns.tolist()
cat_cols = X.select_dtypes(include='object').columns.tolist()
cat_cols = [col for col in cat_cols if col != 'native-country']
capital_cols = ['capital-gain', 'capital-loss']
num_cols = [col for col in num_cols if col not in capital_cols]

In [20]:
cat_cols

['workclass',
 'education',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'gender']

In [21]:
display(X.isnull().sum())

age                  0
workclass         2799
fnlwgt               0
education            0
marital-status       0
occupation        2809
relationship         0
race                 0
gender               0
capital-gain         0
capital-loss         0
hours-per-week       0
native-country     857
dtype: int64

**Country encoding**

In [22]:
countries = X['native-country'].unique()
country_dict = {country: i for i, country in enumerate(countries)}


def encode_country(X):
        encoded_val = pd.Series(X.squeeze()).map(country_dict)
        return pd.DataFrame(encoded_val)

def decode_country(X):
    X = pd.Series(X.squeeze()).map({v: k for k, v in country_dict.items()})
    return X


country_encoder = FunctionTransformer(encode_country, decode_country, check_inverse=False)

In [23]:
print(cat_cols, num_cols)

['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender'] ['age', 'fnlwgt', 'hours-per-week']


**define transformers**

In [24]:
country_pipeline = make_pipeline(
        SimpleImputer(strategy='most_frequent', missing_values=np.nan),
        country_encoder
    )

cat_pipeline = make_pipeline(
        SimpleImputer(strategy='constant', fill_value='unknown'),
        OrdinalEncoder()
    )

num_pipeline = make_pipeline(
        StandardScaler()
    )

capital_pipeline = make_pipeline(
        RobustScaler()
    )

preprocessor = make_column_transformer(
        (country_pipeline, ['native-country']),
        (cat_pipeline, cat_cols),
        (num_pipeline, num_cols),
        (capital_pipeline, capital_cols)
    )



In [26]:
y_encoded = LabelEncoder().fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=1)

In [36]:
print(y_train.shape, X_train.shape)

(39073,) (39073, 13)


In [37]:
X_train

,age,workclass,fnlwgt,education,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country
26051,78,Private,111189,7th-8th,Never-married,Machine-op-inspct,Not-in-family,White,Female,0,0,35,Dominican-Republic
47048,49,Self-emp-inc,122066,Some-college,Divorced,Sales,Not-in-family,White,Male,0,0,25,United-States
33914,62,Self-emp-not-inc,168682,7th-8th,Married-civ-spouse,Sales,Husband,White,Male,0,0,5,United-States
22131,18,Private,110230,10th,Never-married,Other-service,Own-child,White,Male,0,0,11,United-States
46451,40,Private,373050,12th,Married-civ-spouse,Other-service,Husband,White,Male,0,0,40,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43723,41,Local-gov,36924,Some-college,Married-civ-spouse,Adm-clerical,Husband,White,Male,0,0,40,United-States
32511,27,Private,180553,HS-grad,Separated,Machine-op-inspct,Unmarried,White,Female,0,0,40,United-States
5192,18,Private,179597,HS-grad,Never-married,Other-service,Not-in-family,White,Male,0,0,20,United-States
12172,46,Private,145290,Assoc-voc,Married-civ-spouse,Sales,Husband,White,Male,0,0,40,United-States


#### Model

In [38]:
model_pipeline = make_pipeline(
    preprocessor,
    KNeighborsClassifier()
)

# param_grid = {
#     # 'columntransformer__pipeline-1__knnimputer__n_neighbors': [5, 7, 10],
#     #  'randomforestclassifier__n_estimators': [100, 200, 300],
#     "kneighborsclassifier__n_neighbors": range(1,30)
# }

# grid = GridSearchCV(model_pipeline, param_grid, cv=5, error_score='raise')


In [39]:
model_pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('functiontransformer',
                                                                   FunctionTransformer(check_inverse=False,
                                                                                       func=<function encode_country at 0x7f2db44ffac0>,
                                                                                       inverse_func=<function decode_country at 0x7f2edd41d1b0>))]),
                                                  ['native-country']),
                                                 ('pipe...
                                                                   OrdinalEncoder())]),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'gender']),
                                                 ('pipeline-3',
                                                  Pipeline(steps=[('standardscaler',
                                                                   StandardScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'hours-per-week']),
                                                 ('pipeline-4',
                                                  Pipeline(steps=[('robustscaler',
                                                                   RobustScaler())]),
                                                  ['capital-gain',
                                                   'capital-loss'])])),
                ('kneighborsclassifier', KNeighborsClassifier())])

In [40]:
grid.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('pipeline-1',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('functiontransformer',
                                                                                          FunctionTransformer(check_inverse=False,
                                                                                                              func=<function encode_country at 0x7f2db44ffac0>,
                                                                                                              inverse_func=<function decode_country...
                                                                          'occupation',
                                                                          'relationship',
                                                                          'race',
                                                                          'gender']),
                                                                        ('pipeline-3',
                                                                         Pipeline(steps=[('standardscaler',
                                                                                          StandardScaler())]),
                                                                         ['age',
                                                                          'fnlwgt',
                                                                          'hours-per-week']),
                                                                        ('pipeline-4',
                                                                         Pipeline(steps=[('robustscaler',
                                                                                          RobustScaler())]),
                                                                         ['capital-gain',
                                                                          'capital-loss'])])),
                                       ('kneighborsclassifier',
                                        KNeighborsClassifier())]),
             param_grid={'kneighborsclassifier__n_neighbors': range(1, 30)})

**evaluate**

In [33]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

y_pred = grid.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Recall:", recall)
print("F1 Score:", f1)

NotFittedError: This GridSearchCV instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.